In [1]:
from datetime import datetime, timedelta
import pandas as pd
import glob, os
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
import datetime as dt 
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")  
from sklearn.metrics import f1_score
from scipy import stats
from sklearn.metrics import precision_score

In [2]:
# Read layer3-60 data from all the csv files 
path = r'C:/Users/jainsac/Downloads/MRP/Assignment 2/Data/Oct30min/'
Resultpath = r'C:/Users/jainsac/Downloads/MRP/Assignment 2/Results/'
data = pd.concat(map(lambda file: pd.read_csv(file, header = 0,sep='\t', encoding='utf-8'), glob.glob(os.path.join('', path+'/*.csv'))))
#data.columns = ['StartTime', 'EndTime', 'Channel', 'Power_dbm','SNR', 'Occupancy']
data.columns = ['StartTime', 'EndTime', 'Channel', 'Occupancy']
data['StartTime'] = pd.to_datetime(data['StartTime'])
data['EndTime'] = pd.to_datetime(data['EndTime'])

In [3]:
data.head()

,StartTime,EndTime,Channel,Occupancy
0,2016-10-01 05:00:00,2016-10-01 05:30:00,10670,100.0
1,2016-10-01 05:30:00,2016-10-01 06:00:00,10670,100.0
2,2016-10-01 06:00:00,2016-10-01 06:30:00,10670,100.0
3,2016-10-01 06:30:00,2016-10-01 07:00:00,10670,100.0
4,2016-10-01 07:00:00,2016-10-01 07:30:00,10670,100.0


In [4]:
chList = pd.read_csv('C:/Users/jainsac/Downloads/MRP/Assignment 2/ChannelList.csv', header=0, sep ='\t')

In [5]:
channel_list = chList.Channel.unique().tolist()

In [6]:
#channel_list = [16010, 15820, 15540, 15530, 15570, 22490, 23480, 21340, 24220]

In [7]:
# List of channels will override the final list until entire data is available
channel_list = [16010, 15820, 15540, 15530, 15570, 22490, 23480, 21340, 24220, 16130, 24670, 24890, 25410, 25810, 24530,26030, 29310, 28150,
                31020, 27710, 34700, 50132, 50193, 31780, 50175, 50271, 50280, 50269, 50259, 50285, 50309, 50617, 50361, 50295, 50287, 50659, 
                50677, 51028, 50673, 51085, 51095, 51119, 51111, 51224, 51086, 53112, 51292, 51272, 53236, 53144, 10670, 11720, 11710, 10930, 
                2340, 15420, 15990, 50969, 12040, 29990, 11870, 15220, 15130, 13300, 13520, 51366, 53334, 53333, 51176, 51026, 54494, 53970, 
                53379, 54006, 53442, 54620, 56303, 56304, 56293, 56097, 56305, 56306, 56307, 56308, 56333, 56350, 56352, 56353, 56354, 56355, 
                56360, 56356, 56357, 56358, 56359, 1370, 3860, 830, 2020, 131, 11830, 11840, 11860, 9230, 11800, 55232, 55240, 55292, 55293,
                55338, 55355, 55361, 55372, 55410, 55417, 55457, 55481, 55482, 55508, 55510, 55520, 55639, 55640, 55678, 55692, 55713, 55797,
                55858, 55902, 55920, 55922, 56033, 56181, 56230, 56232, 56283, 56294, 56295, 56296, 56300, 56301, 56322, 56323, 56324, 56325,
                56334, 56340, 56341, 56342, 56343
               ]

In [8]:
#TimeInducedDfOriginal = merged

TimeInducedDfOriginal = data

In [9]:
TimeInducedDf = TimeInducedDfOriginal

In [10]:
pValue = []
#for fr in thresh_list:
fr = 80
for ch in channel_list:
    #print(ch)
    #print(fr)


    TimeInducedDf = TimeInducedDfOriginal[(TimeInducedDfOriginal.Channel == ch)][['StartTime', 'EndTime', 'Channel', 'Occupancy']]
    TimeInducedDf['StartTimeIndex'] = TimeInducedDf['StartTime'].values.astype('<M8[m]')
    TimeInducedDf['EndTimeStrip'] = TimeInducedDf['EndTime'].values.astype('<M8[m]')
    TimeInducedDf = TimeInducedDf.drop(['StartTime', 'EndTime'], 1)
    TimeInducedDf = TimeInducedDf[[ 'Channel','StartTimeIndex', 'EndTimeStrip','Occupancy']]
    TimeInducedDf = TimeInducedDf.set_index('StartTimeIndex')
    TimeInducedDf= TimeInducedDf.reindex(pd.date_range(start=TimeInducedDf.index[0], end=TimeInducedDf.index[-1], freq='6h'))
    TimeInducedDf[TimeInducedDf['Occupancy'].isnull()]
    k = 2
    TimeInducedDf['T1'] = TimeInducedDf['Occupancy'].shift()
    while k <=6:
        TimeInducedDf['T'+str(k)] = TimeInducedDf['T'+str(k-1)].shift()
        k=k+1
    # Convert index back to dataframe column
    TimeInducedDf.reset_index(level=0, inplace=True)

    # Remove NAs
    TimeInducedDf = TimeInducedDf.dropna(subset=['Occupancy', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6'], how='any')
    TimeInducedDf['OccupancyClass'] = pd.cut(TimeInducedDf['Occupancy'], [-np.inf, fr, np.inf], labels=[0, 1])

    TimeInducedDFy = TimeInducedDf[(TimeInducedDf.Channel == ch)][['OccupancyClass']] 
    t = TimeInducedDFy.groupby('OccupancyClass').size()
    if (t[0] > 5 and t[1] > 5):
        LRaccuracy = []
        LRsensitivity = []
        LRspecificity = []
        LRfmeasure = []
        LRprecision = []
        NBaccuracy = []
        NBsensitivity = []
        NBspecificity = []
        NBfmeasure = []
        NBprecision = []
        FoldCount = []
        NumCount = []
        channelNum = []
        Threshold = []
        # split the dataframe into two sets: x - attributes, y - response variables
        # in our case, y = Occupancy, x would be rest of the columns
        TimeInducedDFx = TimeInducedDf[(TimeInducedDf.Channel == ch)][['Channel', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6']]

        # Insert a while loop to averager out the runs. value of numofruns = 10
        numOfRuns = 1
        while numOfRuns <= 10:

            numOfSplits = 10   
            # initialize variables for each channel
            n = len(TimeInducedDFx)
            fold = 0
            #for trainRatio in np.arange(0.05, 1, 0.05):
            kf = KFold(n_splits = numOfSplits, shuffle=True, random_state = 777)
            for train_idx, test_idx in kf.split(TimeInducedDFx, TimeInducedDFy):
                X_trainNew, X_testNew = TimeInducedDFx.iloc[train_idx], TimeInducedDFx.iloc[test_idx]
                y_trainNew, y_testNew = TimeInducedDFy.iloc[train_idx], TimeInducedDFy.iloc[test_idx]

                # check if values in y_train are more than 1. 
                # if len(y_trainNew.OccupancyClass.unique()) > 1:
                fold += 1
                FoldCount.append(fold)
                NumCount.append(numOfRuns)

                # Logistic Regression for reduced dataset
                regr = LogisticRegression()                   
                clf2 = regr.fit(X_trainNew, y_trainNew)
                y_predNew = clf2.predict(X_testNew)
                scoreLRRed = clf2.score(X_testNew, y_testNew)
                LRaccuracy.append(scoreLRRed)  
                cfm_LR = confusion_matrix(y_testNew, y_predNew, labels=[0,1])

                # Calculate sensitivity
                sensitivityLR = cfm_LR[1,1]/(cfm_LR[1,0]+cfm_LR[1,1])
                LRsensitivity.append(sensitivityLR)

                # Calculate Specificity
                specificityLR = cfm_LR[0,0]/(cfm_LR[0,0]+cfm_LR[0,1])
                LRspecificity.append(specificityLR)

                # Calculate Precision
                precisionLR = precision_score(y_testNew,y_predNew,average='weighted')
                LRprecision.append(precisionLR)

                # Calculate f-measure
                fmeasureLR = f1_score(y_testNew,y_predNew,average='weighted')
                LRfmeasure.append(fmeasureLR)

                # Naive Bayes for reduced dataset
                modelRed = GaussianNB()

                # Train the model using the training sets 
                modelRed.fit(X_trainNew, y_trainNew)

                #Predict Output 
                predictedRed = modelRed.predict(X_testNew)
                scoreNBRed = accuracy_score(y_testNew, predictedRed)
                NBaccuracy.append(scoreNBRed)

                channelNum.append(ch)
                Threshold.append(fr)

                cfm_NB = confusion_matrix(y_testNew, predictedRed, labels=[0,1])
                # Calculate sensitivity
                sensitivityNB = cfm_NB[1,1]/(cfm_NB[1,0]+cfm_NB[1,1])
                NBsensitivity.append(sensitivityNB)

                # Calculate Specificity
                specificityNB = cfm_NB[0,0]/(cfm_NB[0,0]+cfm_NB[0,1])
                NBspecificity.append(specificityNB)

                # Calculate Precision
                precisionNB = precision_score(y_testNew,predictedRed,average='weighted')
                NBprecision.append(precisionNB)

                # Calculate f-measure
                fmeasureNB = f1_score(y_testNew,predictedRed,average='weighted')
                NBfmeasure.append(fmeasureNB)

            numOfRuns = numOfRuns + 1
        # Add the data to the csv
        plc_df = pd.DataFrame({'Channel': channelNum,'Threshold': Threshold,  
                               'NumCount':NumCount, 'Fold': FoldCount, 
                               'LRaccuracy':LRaccuracy, 'NBaccuracy': NBaccuracy,
                               'LRsensitivity':LRsensitivity,'NBsensitivity':NBsensitivity, 
                               'LRspecificity':LRspecificity,'NBspecificity':NBspecificity,
                               'LRprecision': LRprecision, 'NBprecision':NBprecision,
                               'NBfmeasure':NBfmeasure, 'LRfmeasure':LRfmeasure
                              })
        plc_df.to_csv(Resultpath+'Output-6hours.csv', header=False,sep='\t', encoding='utf-8', index=False, mode = 'a')
        #print(plc_df)
#    if (len(LRaccuracy) > 0 & len(NBaccuracy) > 0):
p = stats.ttest_ind(LRaccuracy,NBaccuracy)
#print(p[1])
pValue.append(p[1])
#print(pValue)

In [11]:
read_plc = pd.read_csv(Resultpath+'Output-6hours.csv', header=1, sep ='\t')

In [12]:
read_plc.columns = ['Channel','Fold','LRaccuracy', 'LRfmeasure', 
                    'LRprecision', 'LRsensitivity', 'LRspecificity', 
                    'NBaccuracy',' NBfmeasure', 'NBsensitivity',
                    'NBspecificity','NBprecision',
                    'NumCount', 'Threshold']

In [13]:
read_plc.head()

,Channel,Fold,LRaccuracy,LRfmeasure,LRprecision,LRsensitivity,LRspecificity,NBaccuracy,NBfmeasure,NBsensitivity,NBspecificity,NBprecision,NumCount,Threshold
0,15540,3,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,1,80
1,15540,4,1.000000,1.000000,1.000000,NaN,1.000000,0.833333,0.909091,1.000000,NaN,0.833333,1,80
2,15540,5,0.818182,0.900000,1.000000,NaN,0.818182,0.727273,0.842105,1.000000,NaN,0.727273,1,80
3,15540,6,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,1,80
4,15540,7,0.909091,0.865801,0.826446,0.0,1.000000,0.909091,0.865801,0.826446,0.0,1.000000,1,80


In [14]:
read_plc = read_plc.drop('NumCount', 1)
read_plc = read_plc.drop('Fold', 1)

In [15]:
ExtendedResultsDF = read_plc.groupby(['Channel','Threshold']).mean().add_suffix('_avg').reset_index()

In [16]:
ExtendedResultsDF.head()

,Channel,Threshold,LRaccuracy_avg,LRfmeasure_avg,LRprecision_avg,LRsensitivity_avg,LRspecificity_avg,NBaccuracy_avg,NBfmeasure_avg,NBsensitivity_avg,NBspecificity_avg,NBprecision_avg
0,1370,80,0.920455,0.904618,0.895422,0.416667,0.978409,0.875000,0.903146,0.948323,0.861111,0.880455
1,2340,80,0.762879,0.746727,0.754543,0.248148,0.889949,0.718182,0.719677,0.747828,0.285185,0.820267
2,10670,80,0.964394,0.964502,0.973199,0.960714,0.971429,0.972727,0.973384,0.978636,0.973214,0.985714
3,10930,80,0.920455,0.919749,0.932795,0.943333,0.899643,0.937879,0.937840,0.948481,0.955000,0.924643
4,11710,80,0.991667,0.991725,0.993056,0.985714,1.000000,0.991667,0.991608,0.992857,1.000000,0.983333


#### Above table indicates average of performance measure per Threshold per channel

In [17]:
# Write the results to a CSV
ExtendedResultsDF.to_csv(Resultpath+'ExtendedResults-6hours.csv', header=False,sep='\t', index=False)

In [18]:
# Create a subset copy with two columns: channels and threshold
ExtendedResultsDFTemp = ExtendedResultsDF[['Channel', 'Threshold']] 

# count the number of channels in each threshold
ExtendedResultsDFTemp = ExtendedResultsDFTemp.groupby(['Threshold']).count().reset_index()

In [19]:
read_plc = read_plc.drop('Channel', 1)

In [20]:
CompactResultsDF = read_plc.groupby(['Threshold']).mean().add_suffix('_avg').reset_index()

In [21]:
# Inner join channels
CompactResultsDFFinal = pd.merge(CompactResultsDF,ExtendedResultsDFTemp, on=['Threshold','Threshold'])

In [22]:
CompactResultsDFFinal['TimeResolution'] = '6hours'
CompactResultsDFFinal['pValue'] = p[1]

In [23]:
CompactResultsDFFinal

,Threshold,LRaccuracy_avg,LRfmeasure_avg,LRprecision_avg,LRsensitivity_avg,LRspecificity_avg,NBaccuracy_avg,NBfmeasure_avg,NBsensitivity_avg,NBspecificity_avg,NBprecision_avg,Channel,TimeResolution,pValue
0,80,0.855949,0.842205,0.849854,0.673719,0.837692,0.832254,0.83304,0.859188,0.720364,0.818288,75,6hours,0.001142


In [24]:
# Write the results to a CSV
CompactResultsDFFinal.to_csv(Resultpath+'CompactResults-Baseline.csv', header=False,sep='\t', index=False, mode = 'a')